In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from itertools import permutations 
import sys
base = '/home3/ebrahim/isr/'
sys.path.append(base)
from datasets import OneHotLetters, OneHotLetters_test
from RNNcell import RNN_one_layer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from skimage.measure import block_reduce
from scipy.stats import pearsonr
import pandas as pd
import seaborn as sns
import pickle
from itertools import permutations, islice
import wandb
device = torch.device("cuda:0")

In [2]:
# initialize untrained model
batch_size = 1
stateful = True
model = RNN_one_layer(28, 200, 28, noise_std=0,
                        feedback_bool=True, bias=False, plastic=False)
model = model.to(device)

In [3]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, 
        weight_decay=0)

loss_fn = torch.nn.CrossEntropyLoss()

In [6]:
# Let's test if the model works 
h_prev_tensor = None
delay_start = 3
delay_middle = 1

# create dataloader
rtt = DataLoader(OneHotLetters(9, 2, '/home3/ebrahim/isr/test_set/test_lists_cleaned_12.pkl', 28, batch_size=batch_size, 
num_letters=26, delay_start=3, delay_middle=1), batch_size=batch_size, shuffle=False)

for batch_idx, (X,y) in enumerate(rtt):

    ll = int((X.shape[1] - (delay_start + delay_middle + 1))/2)

    # Compute prediction and loss
    if batch_idx==0 or stateful==False:
        # init initial states
        y0, h0, hebb0, i0 = model.init_states(1, device, 
        0.5)
    # if stateful is true and not on the first trial
    else:
        y0 = y_hat.detach()
        h0 = h.detach()
        i0 = i.detach()
        hebb0 = hebb.detach()

    loss = 0
    h_single_trial = []

    X = X.to(device)
    y = y.to(device)
    
    # run RNN and compute loss
    for timestep in range(X.shape[1]):

        #if timestep >= delay_start and timestep < (delay_start+ll):
        #    encoding_phase = True
        #else:
        #   encoding_phase = False

        h_single_trial.append(h)

        # initial feedback 
        if timestep == 0:
            y_hat, h, hebb, i = model(X[:, timestep, :], h0, y0, hebb0, i0, device)
        else:
            y_hat, h, hebb, i = model(X[:, timestep, :], h, y[:, timestep-1, :], hebb, i, device)

        #if encoding_phase: 
        #    h = EM_retrieval(h, h_prev_tensor, batch_idx)
        #    h_current_list.append(h.detach())

        loss += loss_fn(y_hat, y[:, timestep, :])

    #h_prev_tensor = torch.stack(h_current_list).squeeze()

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()


print("Model can leave the port!")

NameError: name 'h' is not defined

In [13]:
def EM_retrieval(h, h_prev_tensor, batch_idx):

    # no EM store exists 
    if batch_idx == 0:
        return h

    cos_sim = torch.nn.CosineSimilarity(dim=1)
    cos_sim_values = cos_sim(h, h_prev_tensor)
    softmax = torch.nn.Softmax(dim=0)
    softmax_cos_sim_values = softmax(cos_sim_values*100)
    print(softmax_cos_sim_values.shape)
    print(h_prev_tensor.shape)
    EM = softmax_cos_sim_values@h_prev_tensor
    h_EM = h + EM
    return h_EM 
    

In [ ]:
h_prev_tensor = torch.stack(h_current_list).squeeze()
cos_sim = torch.nn.CosineSimilarity(dim=1)
cos_sim_values = cos_sim(h, h_prev_tensor)
softmax = torch.nn.Softmax(dim=0)
softmax_cos_sim_values = softmax(cos_sim_values*100)
EM = softmax_cos_sim_values@h_prev_tensor
print(EM.shape)




torch.Size([1, 200])
torch.Size([200])


In [52]:
a = torch.Tensor([.25,.25,.5])
b = torch.Tensor([[1,0,0],[0,1,0], [0,0,1]])
print(a.shape)
print(b.shape)
a@b

torch.Size([3])
torch.Size([3, 3])


tensor([0.2500, 0.2500, 0.5000])